In [1]:
#!pip install matplotlib
#!pip install tensorflow


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


#  Preparing Data

In [4]:
import os
# Directory to check for existence
train_dir = '../../train'
# Directory to check for existence
valid_dir = '../../valid'



In [5]:
height,width=180,180



In [6]:
final_height = 180
final_width = 180

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.2),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(
        height_factor=0.1, 
        width_factor=0.1, 
        fill_mode='reflect'
    ),
    # Resizing layer to ensure all images are of the same size
    tf.keras.layers.experimental.preprocessing.Resizing(final_height, final_width),
])


No augmentation

In [15]:

train_set = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(height, width),
    shuffle=True,
)
class_names = train_set.class_names
#train_set = train_set.map(lambda x, y: (data_augmentation(x, training=True), y))

valid_set = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir,
    image_size=(height, width),
)


#batches = 20
#train_set = train_set.take(batches)
#valid_set = valid_set.take(batches)


Found 81946 files belonging to 10 classes.
Found 10244 files belonging to 10 classes.


Augmentation

In [12]:
batch_size = 32

train_set = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(height, width),
    batch_size=batch_size,
    shuffle=True,
)

valid_set = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir,
    image_size=(height, width),
    batch_size=batch_size  # Keep validation batch size the same for consistency
)

train_set = train_set.map(lambda x, y: (data_augmentation(x, training=True), y))

train_set = train_set.prefetch(buffer_size=tf.data.AUTOTUNE)
valid_set = valid_set.prefetch(buffer_size=tf.data.AUTOTUNE)


Found 81946 files belonging to 10 classes.
Found 10244 files belonging to 10 classes.


In [8]:
train_set

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [8]:
valid_set = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,image_size=(height,width))

Found 10244 files belonging to 10 classes.


In [9]:
valid_set

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam

# Load the EfficientNetV2S model without the top layer
base_model = tf.keras.applications.EfficientNetV2S(
    include_top=False,  # Do not include the fully connected layer at the top
    weights="imagenet",  # Use weights pre-trained on ImageNet
    input_tensor=None,  # Optional custom input tensor
    input_shape=None,  # Specify your custom input shape if necessary
    pooling=None,  # Specify this if you need a specific type of pooling
    include_preprocessing=True  # Include preprocessing according to the model's requirements
)

# Make sure the base model is not trainable
base_model.trainable = False

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer for 10 classes
predictions = Dense(10, activation='softmax')(x)

# This is the model we will train
efficient_net_new = Model(inputs=base_model.input, outputs=predictions)

efficient_net_new.compile(optimizer=Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [16]:
def preprocess_image(image, label):
    resized_image = tf.image.resize(image, [384, 384])  # Resize to the expected input size
    return resized_image, label

# Assuming `train_dataset` and `valid_dataset` are your original datasets
train_dataset_resized = train_set.map(preprocess_image)
valid_dataset_resized = valid_set.map(preprocess_image)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    min_delta=0.001,  # Minimum change to qualify as an improvement
    patience=4,  # Stop after 10 epochs without improvement
    verbose=1,  # Print a message when stopping
    mode='min',  # Minimize the monitored quantity (val_loss)
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

# Train the model with the early stopping callback
history = efficient_net_new.fit(
    train_set,
    epochs=30,  # Example number of epochs
    validation_data=valid_set,
    callbacks=[early_stopping]  # Add early stopping to the list of callbacks
)

efficient_net_new.save_weights('efficientnet_augmentation_full_dataset_epochs10.h5')


Epoch 1/30
2561/2561 [==============================] - 1371s 535ms/step - loss: 0.3168 - accuracy: 0.8887 - val_loss: 0.2799 - val_accuracy: 0.9000
Epoch 2/30
2561/2561 [==============================] - 1434s 560ms/step - loss: 0.2480 - accuracy: 0.9121 - val_loss: 0.2769 - val_accuracy: 0.9050
Epoch 3/30
2561/2561 [==============================] - 1383s 540ms/step - loss: 0.2237 - accuracy: 0.9205 - val_loss: 0.2573 - val_accuracy: 0.9104
Epoch 4/30
2561/2561 [==============================] - 1410s 550ms/step - loss: 0.2092 - accuracy: 0.9254 - val_loss: 0.2071 - val_accuracy: 0.9291
Epoch 5/30
2561/2561 [==============================] - 1429s 558ms/step - loss: 0.2013 - accuracy: 0.9289 - val_loss: 0.2046 - val_accuracy: 0.9302
Epoch 6/30
2561/2561 [==============================] - 1438s 561ms/step - loss: 0.1955 - accuracy: 0.9306 - val_loss: 0.2201 - val_accuracy: 0.9290
Epoch 7/30
2561/2561 [==============================] - 1515s 592ms/step - loss: 0.1912 - accuracy: 0.9325